## Dissertação: Frederico Gome

**Objetivo:** Limpeza e processamento de dados


**Autor:** Maria Luiza Campos

**Data:** Julho 2025

### Considerações:

• Todos os dados descritos como "anterior ao ano x" na coluna "Anos dos Fatos" foram alterados pra "não informado" na coluna final "anos_fatos". Isso porque a informação de "ser anterior a algum ano" não condiz com a descrição da coluna "anos_fatos", isto é:

"Ano em que houve a consumação de estupro de vulnerável. Estupro de vulnerável é entendido, neste contexo, como qualquer interação
libidinosa com o menor, desde relações sexuais a carícias."

• Todas as linhas na coluna original "Anos dos Fatos" que constam a informação da data do processo original também foram alterados para
"não informado" na coluna final "anos_fatos". O ano em que o processo iniciou não necessariamente condiz com "anos_fatos". 

• Processo 134780 não encontrado no drive. Coluna dos anos se contradiz. Eu troquei pra ser "Não Informado". 

• Processo 1830642 com informações confusas na coluna de anos. 

In [25]:
## Importando bibliotecas

import pandas as pd ## manipulação de dataframes
import numpy as np  ## operações matemáticas simples e complexas
import re ## processamento de strings

## Importando dados

df_raw = pd.read_excel("rawdata_ajustado.xlsx", engine='openpyxl')
df_raw.head()

,Processo,Ano do fatos,Idade da Vítima,Faixa Etária da Vítima,Idade do Réu,Faixa Etária do Réu,Resultado do julgamento,Vínculo afetivo/relacionamento,Categoria de Vínculo,Coabitação,Filhos entre as Partes,Consentimento Familiar,Consentimento da Vítima,Aplicação da Súmula 593/STJ ou do Tema 918
0,REsp 1480881 - PI (Tema 918/STJ),2010,11 anos,11 a 12 anos,25 anos,20 a 25 anos,Condenado,Namoro,Afetivo,Não,Não,Não,Sim,Sim
1,AgRg no REsp 2033544 - SC,2012,11 anos,11 a 12 anos,31 anos,31 a 35 anos,Condenado,Casual,Afetivo,Não,Não,Não,Sim,Sim
2,AgRg no REsp 2112802 - MG,2017,13 anos,13 a 14 anos,18 anos,18 a 19 anos,Condenado,Namoro,Afetivo,Não,Não,Não,Sim,Sim
3,AgRg no AREsp 2645163 - PR,2017,12 anos,11 a 12 anos,18 anos,18 a 19 anos,Condenado,Namoro,Afetivo,Não,Não,Não,Sim,Sim
4,AgRg no HC 849912 - MG,2018,13 anos,13 a 14 anos,20 anos,20 a 25 anos,Condenado,Namoro,Afetivo,Não,Sim,Não,Sim,Sim


In [26]:
## Vamos começar limpando a primeira coluna

print(df_raw['Processo'].head(5))

0    REsp 1480881 - PI (Tema 918/STJ)
1           AgRg no REsp 2033544 - SC
2           AgRg no REsp 2112802 - MG
3          AgRg no AREsp 2645163 - PR
4              AgRg no HC 849912 - MG
Name: Processo, dtype: object


In [27]:
## Vamos criar uma função para limpar a primeira coluna

def extrair_info_processo(valor):
    if pd.isnull(valor) or not isinstance(valor, str):
        return "não informado", "não informado", "não informado"
    
    # Extrai o número de 6 dígitos
    id_proc_match = re.search(r'(\d{6})', valor)
    id_processo = id_proc_match.group(1) if id_proc_match else "não informado"
    
    # Extrai a classe processual (tudo antes do número)
    classe_match = re.search(r'^(.*?)\s*\d{6}', valor)
    classe = classe_match.group(1).strip() if classe_match else "não informado"
    
    # Extrai o estado (duas letras após hífen)
    estado_match = re.search(r'-\s*([A-Z]{2})\b', valor)
    estado = estado_match.group(1) if estado_match else "não informado"
    
    return id_processo, classe, estado

## aplicando a função

## primeiro, vou criar uma cópia do dataframe original

dados = df_raw.copy()

## agora, aplico a função

dados[['ID_processo', 'classe_processual', 'estado']] = dados['Processo'].apply(
    lambda x: pd.Series(extrair_info_processo(x))
)

##visualizando o resultado

print(dados.head(5))

## organizar as colunas

nova_ordem = ['ID_processo', 'classe_processual', 'estado'] + [col for col in dados.columns if col not in ['ID_processo', 'classe_processual', 'estado']]

dados = dados[nova_ordem]

## agora vamos remover a coluna original

dados = dados.drop(columns=['Processo'])

                           Processo  Ano  do fatos Idade da Vítima  \
0  REsp 1480881 - PI (Tema 918/STJ)           2010         11 anos   
1         AgRg no REsp 2033544 - SC           2012         11 anos   
2         AgRg no REsp 2112802 - MG           2017         13 anos   
3        AgRg no AREsp 2645163 - PR           2017         12 anos   
4            AgRg no HC 849912 - MG           2018         13 anos   

  Faixa Etária da Vítima Idade do Réu Faixa Etária do Réu   \
0           11 a 12 anos      25 anos         20 a 25 anos   
1           11 a 12 anos      31 anos         31 a 35 anos   
2           13 a 14 anos      18 anos         18 a 19 anos   
3           11 a 12 anos      18 anos         18 a 19 anos   
4           13 a 14 anos      20 anos         20 a 25 anos   

  Resultado do julgamento Vínculo afetivo/relacionamento Categoria de Vínculo  \
0               Condenado                         Namoro              Afetivo   
1               Condenado                   

In [28]:
## Agora, vamos limpar a coluna de anos
list(dados.columns)
print(dados.columns)

dados = dados.rename(columns={'Ano  do fatos': 'ano_fatos'})

Index(['ID_processo', 'classe_processual', 'estado', 'Ano  do fatos',
       'Idade da Vítima', 'Faixa Etária da Vítima', 'Idade do Réu',
       'Faixa Etária do Réu ', 'Resultado do julgamento',
       'Vínculo afetivo/relacionamento', 'Categoria de Vínculo', 'Coabitação',
       'Filhos entre as Partes', 'Consentimento Familiar',
       'Consentimento da Vítima',
       'Aplicação da Súmula 593/STJ ou do Tema 918'],
      dtype='object')


In [29]:
## Vamos agora limpar a coluna "Faixa Etária da Vítima"

## Vamos criar uma função para criar categorias

def categorizar_faixa_etaria(valor):
    if isinstance(valor, str) and 'não informado' in valor.lower():
        return 'Não informado'
    # Extrai todos os números da string
    import re
    numeros = [int(n) for n in re.findall(r'\d+', str(valor))]
    if not numeros:
        return 'Não informado'

    idade = max(numeros)  # usa o maior número da faixa como referência  

    if idade < 12:
        return 0  # categoria 0 = criança (<12)
    elif 12 <= idade < 14:
        return 1  # categoria 1 = pré-adolescente (12-13)
    else:
        return 2  # categoria 2 = 14 ou mais
    
# Cria a nova coluna categórica

dados['faixa_etaria_vitima'] = dados['Faixa Etária da Vítima'].apply(categorizar_faixa_etaria)

## vamos reorganizar as colunas

cols = list(dados.columns) ## cria uma lista de colunas
cols.remove('faixa_etaria_vitima') ## remove o nome dessa coluna da lista para reordenar
idx = cols.index('Faixa Etária da Vítima') ## vamos achar o índice dessa coluna
cols.insert(idx + 1, 'faixa_etaria_vitima') ## inserindo uma coluna ao lado da outra
dados = dados[cols] ## aplicando a reordenação

## A forma como estamos organisando a faixa estária da vítima dá origem a uma variável ordinal categórica, com 3 categorias. Isso não segue exatamente o modelo de fisher, que depende
## de duas categorias (não ordinais) para fazer a tabela de contingência 2x2. 

In [30]:
## Dropando algumas colunas...

dados = dados.drop(columns=['Idade da Vítima', 'Faixa Etária da Vítima'])

In [31]:
print(dados.columns)
print(dados['Faixa Etária do Réu '])

Index(['ID_processo', 'classe_processual', 'estado', 'ano_fatos',
       'faixa_etaria_vitima', 'Idade do Réu', 'Faixa Etária do Réu ',
       'Resultado do julgamento', 'Vínculo afetivo/relacionamento',
       'Categoria de Vínculo', 'Coabitação', 'Filhos entre as Partes',
       'Consentimento Familiar', 'Consentimento da Vítima',
       'Aplicação da Súmula 593/STJ ou do Tema 918'],
      dtype='object')
0      20 a 25 anos
1      31 a 35 anos
2      18 a 19 anos
3      18 a 19 anos
4      20 a 25 anos
           ...     
132    20 a 25 anos
133    20 a 25 anos
134    20 a 25 anos
135    18 a 19 anos
136    20 a 25 anos
Name: Faixa Etária do Réu , Length: 137, dtype: object


In [32]:
## Vamos limpar agora a coluna "Faixa Etária do Réu"

## Vamos criar uma função para criar categorias

def categorizar_faixa_etaria_reu(valor):
    if isinstance(valor, str) and 'não informado' in valor.lower():
        return 'Não informado'
    import re
    numeros = [int(n) for n in re.findall(r'\d+', str(valor))]
    if not numeros:
        return 'Não informado'
    maior = max(numeros)
    if maior <= 20:
        return 0
    elif 21 <= maior <= 24:
        return 1
    elif maior >= 25:
        return 2
    else:
        return 'Não Informado'
    
# Cria a nova coluna categórica

dados['faixa_etaria_reu'] = dados['Faixa Etária do Réu '].apply(categorizar_faixa_etaria_reu)

## A forma como estamos organisando a faixa etária do réu dá origem a uma variável ordinal categórica, com 3 categorias. Isso não segue exatamente o modelo de fisher, que depende
## de duas categorias (não ordinais) para fazer a tabela de contingência 2x2. 

In [33]:
## Vamos reordenar

## Há algum erro de digitação na coluna "Faixa Etária do Réu", assim, eu vou criar um objeto para facilitar o código

col_ref = 'Faixa Etária do Réu '

## processo de reordenação

cols = list(dados.columns) ## criando lista
cols.remove('faixa_etaria_reu') ## removendo para reordenar
idx = cols.index(col_ref) ## encontrando o índice
cols.insert(idx + 1, 'faixa_etaria_reu') ## aplicando reordenação
dados = dados[cols] ## aplicando ao drataframe

In [34]:
## Dropando colunas...

dados = dados.drop(columns=['Idade do Réu', 'Faixa Etária do Réu '])

In [35]:
## Vamos limpar agora a coluna do Resultado do Julgamento. 

## Criando a função para criar categorias

def categorizar_resultado(valor):
    if isinstance(valor, str):
        # Remove espaços no início/fim e converte para minúsculas
        valor_limpo = valor.strip().lower()
        # Remove todos os espaços (inclusive no meio, se quiser máxima robustez)
        valor_limpo = valor_limpo.replace(' ', '')
        if valor_limpo == 'absolvido':
            return 0
        elif valor_limpo == 'condenado':
            return 1
    return 'Não Informado'

## aplicando a função

dados['Resultado do julgamento'] = dados['Resultado do julgamento'].apply(categorizar_resultado)

In [36]:
## Ajustes finais

dados = dados.rename(columns={'Resultado do julgamento': 'resultado_julgamento'})

In [37]:
## Ok, agora vamos limpar a categoria de vínculo

## Criando função

def categorizar_vinculo(valor):
    if isinstance(valor, str):
        valor_limpo = valor.strip().lower()
        if 'afetivo' in valor_limpo:
            return 1
        elif 'indefinido' in valor_limpo:
            return 'Indefinido'
        else:
            return 0
    return 0

## Aplicando a função

dados['Categoria de Vínculo'] = dados['Categoria de Vínculo'].apply(categorizar_vinculo)

## Renomeando a coluna

dados = dados.rename(columns={'Categoria de Vínculo': 'categoria_vinculo'})



In [38]:
## vamos renomear a coluna descrevendo o tipo de vinculo para um nome mais fácil de trabalhar

dados = dados.rename(columns={'Vínculo afetivo/relacionamento': 'descricao_vinculo'})

In [ ]:
## Vamos limpar agora a coluna de coabitação

## criando a função

def categorizar_coabitacao(valor):
    if isinstance(valor, str):
        valor_limpo = valor.strip().lower()
        if valor_limpo == 'sim':
            return 1
        elif valor_limpo == 'não' or valor_limpo == 'nao':
            return 0
        elif valor_limpo == 'não identificado' or valor_limpo == 'nao identificado':
            return 'Não identificado'
    return valor  # Mantém o valor original caso não seja string ou não se encaixe nas opções

## Aplicando a funçao na coluna 

dados['Coabitação'] = dados['Coabitação'].apply(categorizar_coabitacao)

## renomeando a coluna

dados = dados.rename(columns={'Coabitação': 'coab'})

In [41]:
## não está rodando

dados['coab'] = dados['coab'].replace('N/I', 'não identificado')

In [42]:
## Vamos agora limpar a coluna de filhos

## criando a função

def categorizar_filhos(valor):
    if isinstance(valor, str):
        valor_limpo = valor.strip().lower()
        if valor_limpo == 'sim':
            return 1
        elif valor_limpo == 'não' or valor_limpo == 'nao':
            return 0
        elif valor_limpo in ['não informado', 'nao informado', 'não mencionado', 'nao mencionado']:
            return valor  # Mantém o valor original
    return valor  # Mantém o valor original caso não seja string ou não se encaixe nas opções

## aplicando a função 

dados['Filhos entre as Partes'] = dados['Filhos entre as Partes'].apply(categorizar_filhos)

## renomeando coluna

dados = dados.rename(columns={'Filhos entre as Partes': 'filhos'})

In [48]:
## não está rodando

dados['filhos'] = dados['filhos'].replace('N/I', 'não identificado')

In [43]:
## Limpando a coluna de consentimento familiar

## criando a função

def categorizar_consentimento_familiar(valor):
    if isinstance(valor, str):
        valor_limpo = valor.strip().lower()
        if valor_limpo == 'sim':
            return 1
        elif valor_limpo == 'não' or valor_limpo == 'nao':
            return 0
        elif valor.strip() == 'Não Informado':
            return valor  # Mantém exatamente "Não Informado"
    return valor  # Mantém o valor original caso não seja string ou não se encaixe nas opções

## aplicando a função

dados['Consentimento Familiar'] = dados['Consentimento Familiar'].apply(categorizar_consentimento_familiar)

## renomeando coluna

dados = dados.rename(columns={'Consentimento Familiar': 'consentimento_familiar'})

In [50]:
## não está rodando

dados['consentimento_familiar'] = dados['consentimento_familiar'].replace('N/I', 'não identificado')

In [ ]:
## Limpando a coluna de consentimento da vítima

## criando a função

def categorizar_consentimento_vitima(valor):
    if isinstance(valor, str):
        valor_limpo = valor.strip().lower()
        if valor_limpo == 'sim':
            return 1
        elif valor_limpo == 'não' or valor_limpo == 'nao':
            return 0
        elif valor.strip() == 'Não Informado':
            return valor  # Mantém exatamente "Não Informado"
    return valor  # Mantém o valor original caso não seja string ou não se encaixe nas opções

## aplicando a função

dados['Consentimento da Vítima'] = dados['Consentimento da Vítima'].apply(categorizar_consentimento_familiar)

## renomeando coluna

dados = dados.rename(columns={'Consentimento da Vítima': 'consentimento_vitima'})

In [51]:
## não está rodando

dados['consentimento_vitima'] = dados['consentimento_vitima'].replace('N/I', 'não identificado')

In [45]:
### Vamos limpar a variável de aplicação da súmula

## criando a função

def simplificar_sumula(valor):
    if isinstance(valor, str) and 'sim' in valor.lower():
        return 1
    else:
        return 0
    
## aplicando a função

dados['Aplicação da Súmula 593/STJ ou do Tema 918'] = dados['Aplicação da Súmula 593/STJ ou do Tema 918'].apply(simplificar_sumula)

## renomeando coluna

dados = dados.rename(columns={'Aplicação da Súmula 593/STJ ou do Tema 918': 'aplicacao_sumula'})


In [52]:
## Vou checar a natureza dos dados em cada coluna dado que isso pode perturbar a análise dos dados.

print(dados.dtypes)

## ## Vamos substituit "Não informado" por NaN, dado que quero fazer operações matemáticas com essas colunas

dados['faixa_etaria_vitima'] = dados['faixa_etaria_vitima'].replace('Não informado', np.nan).astype(float)
dados['faixa_etaria_reu'] = dados['faixa_etaria_reu'].replace('Não informado', np.nan).astype(float)
dados['categoria_vinculo'] = dados['categoria_vinculo'].replace('Indefinido', np.nan).astype(float)
dados['coab'] = dados['coab'].replace('não identificado', np.nan).astype(float)
dados['filhos'] = dados['filhos'].replace(
    to_replace=r'(?i)^não identificado', value=np.nan, regex=True
).astype(float)
dados['consentimento_familiar'] = dados['consentimento_familiar'].replace('não identificado', np.nan).astype(float)
dados['consentimento_vitima'] = dados['consentimento_vitima'].replace('não identificado', np.nan).astype(float)

## printando de novo

print(dados.dtypes)



ID_processo                object
classe_processual          object
estado                     object
ano_fatos                   int64
faixa_etaria_vitima       float64
faixa_etaria_reu          float64
resultado_julgamento        int64
descricao_vinculo          object
categoria_vinculo         float64
coab                      float64
filhos                    float64
consentimento_familiar     object
consentimento_vitima        int64
aplicacao_sumula            int64
dtype: object
ID_processo                object
classe_processual          object
estado                     object
ano_fatos                   int64
faixa_etaria_vitima       float64
faixa_etaria_reu          float64
resultado_julgamento        int64
descricao_vinculo          object
categoria_vinculo         float64
coab                      float64
filhos                    float64
consentimento_familiar    float64
consentimento_vitima      float64
aplicacao_sumula            int64
dtype: object


/var/folders/19/fl7mdk7d6bs0hjb2wl3v43pm0000gn/T/ipykernel_6402/380911037.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados['consentimento_familiar'] = dados['consentimento_familiar'].replace('não identificado', np.nan).astype(float)


In [53]:
## Ajustando a natureza dos dados

## ID_processo como número

dados['ID_processo'] = pd.to_numeric(dados['ID_processo'], errors='coerce').astype('Int64') ## ID_processo como número

## Colunas tipo string

dados['classe_processual'] = dados['classe_processual'].astype(str)
dados['estado'] = dados['estado'].astype(str)
dados['ano_fatos'] = dados['ano_fatos'].astype(str)

## Coluna com 3 categorias: faixa_etaria_reu

dados['faixa_etaria_reu'] = pd.to_numeric(dados['faixa_etaria_reu'], errors='coerce').astype('Int64')

## Colunas binárias

colunas_binarias = [
    'faixa_etaria_vitima', 'resultado_julgamento', 'coab', 'categoria_vinculo',
    'filhos', 'consentimento_familiar', 'consentimento_vitima', 'aplicacao_sumula'
]

for col in colunas_binarias:
    dados[col] = pd.to_numeric(dados[col], errors='coerce').astype('Int64')

## printando de novo

print(dados.dtypes)

ID_processo                Int64
classe_processual         object
estado                    object
ano_fatos                 object
faixa_etaria_vitima        Int64
faixa_etaria_reu           Int64
resultado_julgamento       Int64
descricao_vinculo         object
categoria_vinculo          Int64
coab                       Int64
filhos                     Int64
consentimento_familiar     Int64
consentimento_vitima       Int64
aplicacao_sumula           Int64
dtype: object


In [54]:
## Salvando o dataset limpo no repositório 

dados.to_csv('dados_limpos.csv', index=False)